# Load the MLFlow model locally and try predictions

## Prerequisites

1. You need to have run successfully the training notebook related to this model, available in this same folder, where at the end of the notebook, after training the model, it downloads the 'artifacts' with the MLFlow model folder ("./artifact_downloads/outputs/mlflow-model").

2. Create a conda environment with the 'conda.yaml' file provided within the "mlflow-model" folder, doing like the following:

   1. (base) /> conda env create --file conda.yaml --name automl-model-classification-env
   
   1. (base) /> conda activate automl-model-classification-env
   
   1. (automl-model-classification-env) /> conda install jupyter nb_conda

3. Run Jupyter and make sure you are using the related 'automl-model-classif-env' Kernel.

4. Run this notebook.

If the MLFlow model files were downloaded successfully by the training notebook, you should see the files here.

In [1]:
import os

# Show the contents of the MLFlow model folder
os.listdir("./artifact_downloads/outputs/mlflow-model")

# You should see a list of files such as the following:
# ['conda.yaml', 'MLmodel', 'model.pkl', 'requirements.txt']

['conda.yaml', 'MLmodel', 'model.pkl', 'requirements.txt']

### Load the test data into a Pandas DataFrame

Load a test dataset (bank_marketing_test_data.csv) into a Pandas DataFrame in order to try some predictions with it.

In [21]:
import pandas as pd

test_df = pd.read_csv(
    "./data/test-mltable-folder/nyc_energy_test_clean.csv"
).reset_index(drop=True)
y_actual = test_df.pop("demand").values

test_df.shape, y_actual.shape

((875, 3), (875,))

## Load the best model in memory

Load the model using MLflow flavor. Check MLmodel under the downloaded folder (artifact_downloads/outputs/mlflow-model).
For this particular example (and for AutoML tabular scenario except for forecast tcn which has Pytorch flavor), MLmodel file will describe two flavors, sklearn and python_function.
We show how to load model using sklearn for forecast models, which predict with .forecast() and .forecast_quantiles().
For more information on MLflow flavors, visit: https://www.mlflow.org/docs/latest/models.html#storage-format

Loading the models locally assume that you are running the notebook in an environment compatible with the model. The list of dependencies that is expected by the model is specified in the MLflow model produced by AutoML (in the 'conda.yaml' file within the mlflow-model folder).

Load the model using MLFlow PyFunc:

In [22]:
import mlflow.pyfunc
import mlflow.sklearn

In [23]:
# Get the MLFlow model from the downloaded MLFlow model files

model = mlflow.sklearn.load_model("./artifact_downloads/outputs/mlflow-model")

In [26]:
# Forecasting models predict with .forecast() or .forecast_quantiles(), not with .predict()

y_preds = model.forecast(test_df, ignore_data_errors=True)

y_preds

# Original forecasting with .forecast_quantiles(X_test)
# https://github.com/Azure/azureml-examples/blob/main/python-sdk/tutorials/automl-with-azureml/forecasting-energy-demand/forecasting_script.py

C:\ProgramData\Miniconda3\envs\automl-model-classif-env\lib\site-packages\azureml\automl\runtime\shared\model_wrappers.py:4074: UserWarning: Actual values are present for all times in the input dataframe - there is nothing to forecast. Please set 'y' values to np.NaN for times where you need a forecast.
  warnings.warn(error.error_message)


(array([     nan,      nan,      nan,      nan,      nan,      nan,
             nan,      nan,      nan,      nan,      nan,      nan,
        5216.917, 4968.842, 4706.283, 4477.242, 4314.367, 4233.333,
        4207.042, 4254.458, 4367.075, 4519.1  , 4776.7  , 5057.075,
        5269.592, 5426.208, 5511.033, 5557.25 , 5526.233, 5502.408,
        5522.642, 5584.158, 5643.492, 5725.4  , 5720.867, 5578.958,
        5330.483, 4996.2  , 4696.283, 4487.033, 4357.758, 4293.558,
        4327.783, 4560.142, 5088.309, 5641.708, 6023.925, 6285.1  ,
        6442.95 , 6468.317, 6396.35 , 6341.342, 6293.525, 6241.8  ,
        6240.008, 6224.3  , 6061.483, 6069.358, 6047.717, 5831.95 ,
        5481.925, 5046.367, 4661.783, 4416.808, 4280.475, 4204.833,
        4233.867, 4481.325, 5003.483, 5579.408, 5942.25 , 6171.858,
        6287.692, 6384.317, 6411.85 , 6425.308, 6392.992, 6387.5  ,
        6384.6  , 6428.933, 6358.325, 6292.675, 6172.633, 5922.9  ,
        5554.017, 5114.008, 4720.575, 4472.2  , 

# Next Steps
You can see further examples of other AutoML tasks such as Image-Classification, Image-Object-Detection, NLP-Text-Classification, Time-Series-Forcasting, etc.